<a href="https://colab.research.google.com/github/H-jamesR2/c0gniz4nt-project-genAI/blob/main/_Z_RegularProjects/generative-ai-with-azure-machine-learning/Building_an_AI_Solution/Building_an_AI_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download dependencies
!pip install datasets numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.7 MB/s eta 0:00:00


In [2]:
!pip install kagglehub

In [3]:
import os
import bz2
import urllib.request
import random
import torch
import numpy as np
from datasets import Dataset
#from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import requests

In [4]:
import kagglehub

In [5]:
# Set seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [6]:
# Download dataset using kagglehub
dataset_path = kagglehub.dataset_download("bittlingmayer/amazonreviews")

# Ensure the directory exists
os.makedirs("amazon_sentiment", exist_ok=True)

# Move downloaded files to desired location
train_file = os.path.join(dataset_path, "train.ft.txt.bz2")
test_file = os.path.join(dataset_path, "test.ft.txt.bz2")

os.rename(train_file, "amazon_sentiment/train.ft.txt.bz2")
os.rename(test_file, "amazon_sentiment/test.ft.txt.bz2")

print("Dataset downloaded and moved to 'amazon_sentiment/' folder.")

100%|██████████| 493M/493M [00:06<00:00, 74.5MB/s]

Extracting files...


Dataset downloaded and moved to 'amazon_sentiment/' folder.


In [7]:
def extract_bz2(file_path):
    with bz2.BZ2File(file_path, "r") as f:
        return [line.decode("utf-8").strip() for line in f]

# Extract train & test data
train_data = extract_bz2("amazon_sentiment/train.ft.txt.bz2")
test_data = extract_bz2("amazon_sentiment/test.ft.txt.bz2")

In [8]:
def parse_fasttext_format(data):
    parsed_data = []

    for line in data:
        parts = line.split(" ", 1)  # Split into label and text
        if len(parts) != 2:
            continue  # Skip invalid lines
        label, text = parts
        label = 0 if label == "__label__1" else 1  # Convert labels to 0/1
        parsed_data.append({"text": text, "label": label})

    return parsed_data

# Convert train/test data
train_data_parsed = parse_fasttext_format(train_data)
test_data_parsed = parse_fasttext_format(test_data)

In [9]:
# Function to balance and downsample
def balance_dataset(data, target_size):
    # Separate positive and negative samples
    positive_samples = [item for item in data if item['label'] == 1]
    negative_samples = [item for item in data if item['label'] == 0]

    print(f"Original positive samples: {len(positive_samples)}")
    print(f"Original negative samples: {len(negative_samples)}")

    # Target half of the size for each class
    half_size = target_size // 2

    # Ensure we don’t exceed available samples
    num_positive = min(half_size, len(positive_samples))
    num_negative = min(half_size, len(negative_samples))

    # Randomly sample from each class
    balanced_positive = random.sample(positive_samples, num_positive)
    balanced_negative = random.sample(negative_samples, num_negative)

    # Combine and shuffle
    balanced_data = balanced_positive + balanced_negative
    random.shuffle(balanced_data)

    return balanced_data

# Apply to train and test data
train_target_size = 18000
test_target_size = 2000

balanced_train_data = balance_dataset(train_data_parsed, train_target_size)
balanced_test_data = balance_dataset(test_data_parsed, test_target_size)

Original positive samples: 1800000
Original negative samples: 1800000
Original positive samples: 200000
Original negative samples: 200000


In [10]:
# Convert to Dataset
train_dataset = Dataset.from_list(balanced_train_data)
test_dataset = Dataset.from_list(balanced_test_data)

#train_dataset = Dataset.from_list(train_data_parsed)
#test_dataset = Dataset.from_list(test_data_parsed)

# Quick check
print(train_dataset[0])

{'text': "hard slogging: I enjoyed pompeii, but I am finding this one hard slogging. The story seems to be a straight narration of political events from one viewpoint, the slave Tiro, and there's very little personal interaction, very little that makes you like or identify with any of the characters, or even hate cicero's nemesis. I got about half way through it and realized it was never going to pick up or make me care about the characters, one way or another and skimmed the rest.", 'label': 0}


In [11]:
# Check balance
train_positive = sum(1 for item in train_dataset if item['label'] == 1)
train_negative = sum(1 for item in train_dataset if item['label'] == 0)
print(f"Train - Positive: {train_positive}, Negative: {train_negative}")
# Send first 10 reviews to Azure model
print(test_dataset[:10])
#for review in test_dataset[:10]:
#for review in test_dataset[:10]['text']:
#    print(review)
#print(test_dataset[:5]['text'])

Train - Positive: 9000, Negative: 9000
{'text': ["ZooTycoon Rocks!!: Zoo Tycoon is the best computer game i've ever had! It keeps me busy for hours and I never get bored. With expansion packs it's even more fun. I would suggest it for anyone who likes animals and is interested in zoos. You also learn about animals and what they like in their exhibits. The graphics are really good and the sounds are cool too. It's a little hard when you start out but it gets easier. It's the best game ever!", "Best book I've ever read: I read this book when I was very small.I was highly impressed with it.I am 23 now and still I'm in love with it,I joined YSS\\SRF 8 years back and have been initiated with Kriya Yoga. Read this book and fall in love with GOD and GURUJI(Paramahansa Yogananda). MAY GOD BLESS US ALL.", 'Good Value: I needed a component cable from my DVD to my ceiling mounted projector. The cables look and feel like good quality materials. This is actually video and audio cable but I used it 

In [14]:
import requests
import json
import urllib.request

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

API_URL = "Azure-generated-endpoint/score"  # Replace with your actual Azure endpoint
api_key = "APIKEY=replace"
HEADERS = {'Content-Type':'application/json',
           'Accept': 'application/json',
           "Authorization": ("Bearer " + "APIKEY=replace")}  # Replace with actual token

headers = {'Content-Type':'application/json', 'Accept': 'application/json', 'Authorization':('Bearer '+ api_key)}


# Function to send text to Azure model API
def get_sentiment(text):
    payload = {"inputs": text}
    try:
        req = urllib.request.Request(API_URL, data=json.dumps(payload).encode(), headers=HEADERS)
        response = urllib.request.urlopen(req)
        return json.loads(response.read().decode())  # Convert byte string to JSON
    except Exception as e:
        print(f"Error: {e}")
        return None

# Convert API response to sentiment labels
def convert_label(pred):
    if pred:
        label = pred[0]["label"]
        return 1 if "5 star" in label or "4 star" in label else 0  # 4-5 stars = Positive, 1-2 stars = Negative
    return None

# Extract text reviews and ground-truth labels
reviews = [item["text"] for item in test_dataset]
actual_labels = [item["label"] for item in test_dataset]

# Get Azure predictions
predictions = [get_sentiment(review) for review in reviews]
predicted_labels = [convert_label(pred) for pred in predictions if pred is not None]

# Ensure lengths match (skip None responses)
min_len = min(len(actual_labels), len(predicted_labels))
actual_labels, predicted_labels = actual_labels[:min_len], predicted_labels[:min_len]

# Compute evaluation metrics
accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.9360
Precision: 0.9217
Recall: 0.9530
F1-score: 0.9371


In [ ]:
# RANDOM
# RANDOM
# RANDOM
# RANDOM

In [ ]:
# Load tokenizer and model (DistilBERT for efficiency)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# Load pre-trained model for sentiment classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Training configuration
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
)

# Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.170000,0.182514
2,0.125500,0.206504
3,0.056800,0.249470


TrainOutput(global_step=3375, training_loss=0.13588959517302338, metrics={'train_runtime': 1294.8268, 'train_samples_per_second': 41.704, 'train_steps_per_second': 2.607, 'total_flos': 3576619763712000.0, 'train_loss': 0.13588959517302338, 'epoch': 3.0})

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_distilbert")
tokenizer.save_pretrained("./fine_tuned_distilbert")

('./fine_tuned_distilbert/tokenizer_config.json',
 './fine_tuned_distilbert/special_tokens_map.json',
 './fine_tuned_distilbert/vocab.txt',
 './fine_tuned_distilbert/added_tokens.json',
 './fine_tuned_distilbert/tokenizer.json')

In [ ]:
import shutil
import os

# Create a zip file of the directory
shutil.make_archive('fine_tuned_distilbert', 'zip', './fine_tuned_distilbert')

# Download the zip file to your local computer
from google.colab import files
files.download('fine_tuned_distilbert.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 0.24947042763233185,
 'eval_runtime': 14.1925,
 'eval_samples_per_second': 140.92,
 'eval_steps_per_second': 8.807,
 'epoch': 3.0}

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
import evaluate
from datasets import load_dataset

# Load accuracy metric from evaluate library
accuracy_metric = evaluate.load("accuracy")

# Define the evaluation function
def compute_metrics(pred):
    # Predictions and labels
    predictions, labels = pred
    predictions = predictions.argmax(axis=-1)  # Get the predicted labels
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Set the compute_metrics function for the trainer
trainer.compute_metrics = compute_metrics

# Evaluate the model
evaluation_results = trainer.evaluate()

print(f"Evaluation Results: {evaluation_results}")

Evaluation Results: {'eval_loss': 0.24947042763233185, 'eval_accuracy': 0.9425, 'eval_runtime': 14.405, 'eval_samples_per_second': 138.841, 'eval_steps_per_second': 8.678, 'epoch': 3.0}


In [ ]:
## Save @ _Z_RegularProjects/generative-ai-with-azure-machine-learning/Implementing_Cloud_Based_NLP_with_Transformers

In [ ]:
# Create DataLoader for validation set
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to predict sentiment on validation set
def predict_validation_set(model, dataloader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)  # Get predicted class

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return predictions, true_labels

# Get predictions on test set
preds, true_labels = predict_validation_set(model, test_dataloader)

# Compute Accuracy
accuracy = np.mean(np.array(preds) == np.array(true_labels))
print(f"Validation Accuracy: {accuracy:.4f}")

# ==============================
# 🚀 Inference Function
# ==============================
def predict_sentiment(review_text):
    model.eval()
    inputs = tokenizer(review_text, return_tensors="pt", truncation=True, padding="max_length", max_length=256)

    # Move tensors to GPU if available
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(output.logits, dim=1).cpu().item()

    sentiment = "Positive" if prediction == 1 else "Negative"
    return sentiment

# Example predictions
sample_reviews = [
    "This product is amazing! I love it.",
    "Worst purchase ever. Total waste of money.",
    "It's okay, not great but not bad."
]

for review in sample_reviews:
    print(f"Review: {review} | Sentiment: {predict_sentiment(review)}")